In [1]:
import numpy as np
import pandas as pd

from utils import dt2int
from torchdiffeq import odeint
from collections import namedtuple
from scipy.integrate import solve_ivp

from data import C3RData, interpolate
from sklearn.metrics import mean_squared_error, mean_squared_log_error

In [2]:
# required data structures
Action = namedtuple("patient_action", ['CHO', 'insulin'])

In [3]:
# load dataset
c3r = C3RData(data_dir="data/")

In [4]:
# computed parameter sets
pEVO = [
    1.2433461765211888,
    0.23746704633494647,
    0.07426,
    0.013128200000000001,
    0.007361601609188121,
    0.08095999996738135,
    0.040101988636679,
    0.01692058325659032,
    0.002499999999999984,
    7.469604659648978,
]

pPSO = [
    1.74453,
    0.17448400259928334,
    0.0009459666709618961,
    0.68523,
    0.012269333333333333,
    0.04857599999999944,
    0.0240612,
    0.07163696962095331,
    0.00016666666666666815,
    6.970227880931038
]

pPINN = [
    0.021419000001136793,
    0.05816134190741184,
    0.0028379,
    0.065641,
    0.036808,
    0.0029398,
    0.008020399994634077,
    0.006121641359098722,
    0.0005,
    3.56213180523,
]

In [5]:
# load test data
test_from_dt = f"2014-05-09 00:00:00"
test_to_dt = f"2014-05-11 00:00:00"

# destructure data
test_cgm_data, test_meal_data, test_insulin_data = c3r.get_case(2, test_from_dt, test_to_dt)
test_data_t, test_data_CGM, test_data_CHO, test_data_insulin = interpolate(test_cgm_data, test_meal_data, test_insulin_data)

# normalize time
test_data_t -= test_data_t[0]

# scale down to minutes
test_data_t = test_data_t / 60.0

# negate delay
test_data_CHO = np.roll(test_data_CHO, -200)
test_data_insulin = np.roll(test_data_insulin, -200)

# clean
test_data_CHO = np.nan_to_num(test_data_CHO, nan=test_data_CHO[0])
test_data_insulin = np.nan_to_num(test_data_insulin, nan=test_data_insulin[0])

In [6]:
# load test2 data
test2_from_dt = f"2014-05-12 00:00:00"
test2_to_dt = f"2014-05-14 00:00:00"

# destructure data
test2_cgm_data, test2_meal_data, test2_insulin_data = c3r.get_case(2, test2_from_dt, test2_to_dt)
test2_data_t, test2_data_CGM, test2_data_CHO, test2_data_insulin = interpolate(test2_cgm_data, test2_meal_data, test2_insulin_data)

# normalize time
test2_data_t -= test2_data_t[0]

# scale down to minutes
test2_data_t = test2_data_t / 60.0

# negate delay
test2_data_CHO = np.roll(test2_data_CHO, -200)
test2_data_insulin = np.roll(test2_data_insulin, -200)

# clean
test2_data_CHO = np.nan_to_num(test2_data_CHO, nan=test2_data_CHO[0])
test2_data_insulin = np.nan_to_num(test2_data_insulin, nan=test2_data_insulin[0])

In [7]:
# load parameters
orig_params = pd.read_csv("parameters/vpatient_params.csv")
mean_params = orig_params.loc[orig_params["Name"] == "adult#001"].squeeze()
mean_params = mean_params.rename(lambda x: x.replace(" ", ""))
adult_params = orig_params[orig_params.Name.str.contains("adult")]

In [8]:
def ode(t, x, p, data_t, data_CHO, data_insulin):
    in_time = data_t
    in_CHO = data_CHO / 2
    in_insulin = np.where(data_insulin > data_insulin[0], data_insulin / 2, data_insulin)

    # instantiate derivatives
    dxdt = np.zeros(13)

    # destructure params
    kabs, kmax, kmin, k2, k1, ki, kp2, kp3, ke1, kp1 = p

    # fixed parameters
    params = mean_params

    # get current action
    t_mask = np.abs(in_time - t).argmin()
    action = Action(CHO=in_CHO[t_mask], insulin=in_insulin[t_mask])

    d = action.CHO * 1000 # g -> mg
    insulin = action.insulin * 6000 / params.BW

    # glucose in the stomach
    qsto = x[0] + x[1]

    last_foodtaken = 0
    if t_mask > 0:
        last_foodtaken = in_CHO[t_mask - 1]
    
    Dbar = last_foodtaken * 1000 # unit: mg

    # stomach solid
    dxdt[0] = -kmax * x[0] + d

    if Dbar > 0:
        aa = 5 / 2 / (1 - params.b) / Dbar
        cc = 5 / 2 / params.d / Dbar
        kgut = kmin + (kmax - kmin) / 2 * (
            np.tanh(aa * (qsto - params.b * Dbar))
            - np.tanh(cc * (qsto - params.d * Dbar))
            + 2
        )
    else:
        kgut = kmax

    # stomach liquid
    dxdt[1] = kmax * x[0] - x[1] * kgut

    # intestine
    dxdt[2] = kgut * x[1] - kabs * x[2]

    # Rate of appearance
    Rat = params.f * kabs * x[2] / params.BW
    # Glucose Production
    EGPt = kp1 - kp2 * x[3] - kp3 * x[8]
    # Glucose Utilization
    Uiit = params.Fsnc

    # renal excretion
    if x[3] > params.ke2:
        Et = ke1 * (x[3] - params.ke2)
    else:
        Et = 0

    # glucose kinetics
    # plus dextrose IV injection input u[2] if needed
    dxdt[3] = max(EGPt, 0) + Rat - Uiit - Et - k1 * x[3] + k2 * x[4]
    dxdt[3] = (x[3] >= 0) * dxdt[3]

    Vmt = params.Vm0 + params.Vmx * x[6]
    Kmt = params.Km0
    Uidt = Vmt * x[4] / (Kmt + x[4])
    dxdt[4] = -Uidt + k1 * x[3] - k2 * x[4]
    dxdt[4] = (x[4] >= 0) * dxdt[4]

    # insulin kinetics
    dxdt[5] = (
        -(params.m2 + params.m4) * x[5]
        + params.m1 * x[9]
        + params.ka1 * x[10]
        + params.ka2 * x[11]
    )  # plus insulin IV injection u[3] if needed
    It = x[5] / params.Vi
    dxdt[5] = (x[5] >= 0) * dxdt[5]

    # insulin action on glucose utilization
    dxdt[6] = -params.p2u * x[6] + params.p2u * (It - params.Ib)

    # insulin action on production
    dxdt[7] = -ki * (x[7] - It)

    dxdt[8] = -ki * (x[8] - x[7])

    # insulin in the liver (pmol/kg)
    dxdt[9] = -(params.m1 + params.m30) * x[9] + params.m2 * x[5]
    dxdt[9] = (x[9] >= 0) * dxdt[9]

    # subcutaneous insulin kinetics
    dxdt[10] = insulin - (params.ka1 + params.kd) * x[10]
    dxdt[10] = (x[10] >= 0) * dxdt[10]

    dxdt[11] = params.kd * x[10] - params.ka2 * x[11]
    dxdt[11] = (x[11] >= 0) * dxdt[11]

    # subcutaneous glucose
    dxdt[12] = -params.ksc * x[12] + params.ksc * x[3]
    dxdt[12] = (x[12] >= 0) * dxdt[12]

    return dxdt

# Test Data 1

In [9]:
# initial conditions
x_0 = mean_params.iloc[2:15].to_numpy()

# timespan
tspan = (test_data_t[0], test_data_t[-1])
t_eval = test_data_t

In [10]:
solEVO = solve_ivp(ode, tspan, x_0, t_eval=t_eval, args=(pEVO, test_data_t, test_data_CHO, test_data_insulin))
solPSO = solve_ivp(ode, tspan, x_0, t_eval=t_eval, args=(pPSO, test_data_t, test_data_CHO, test_data_insulin))
solPINN = solve_ivp(ode, tspan, x_0, t_eval=t_eval, args=(pPINN, test_data_t, test_data_CHO, test_data_insulin))

predEVO = solEVO.y[12] / mean_params["Vg"]
predPSO = solPSO.y[12] / mean_params["Vg"]
predPINN = solPINN.y[12] / mean_params["Vg"]

In [11]:
lEVO = mean_squared_log_error(predEVO, test_data_CGM)
lPSO = mean_squared_log_error(predPSO, test_data_CGM)
lPINN = mean_squared_log_error(predPINN, test_data_CGM)

In [12]:
lEVO, lPSO, lPINN

(0.3266100884733184, 0.3137920963176523, 0.1549490301866874)

# Test Data 2

In [13]:
# initial conditions
x_0 = mean_params.iloc[2:15].to_numpy()

# timespan
tspan = (test2_data_t[0], test2_data_t[-1])
t_eval = test2_data_t

In [14]:
solEVO = solve_ivp(ode, tspan, x_0, t_eval=t_eval, args=(pEVO, test2_data_t, test2_data_CHO, test2_data_insulin))
solPSO = solve_ivp(ode, tspan, x_0, t_eval=t_eval, args=(pPSO, test2_data_t, test2_data_CHO, test2_data_insulin))
solPINN = solve_ivp(ode, tspan, x_0, t_eval=t_eval, args=(pPINN, test2_data_t, test2_data_CHO, test2_data_insulin))

predEVO = solEVO.y[12] / mean_params["Vg"]
predPSO = solPSO.y[12] / mean_params["Vg"]
predPINN = solPINN.y[12] / mean_params["Vg"]

In [15]:
lEVO = mean_squared_log_error(predEVO, test_data_CGM)
lPSO = mean_squared_log_error(predPSO, test_data_CGM)
lPINN = mean_squared_log_error(predPINN, test_data_CGM)

In [16]:
lEVO, lPSO, lPINN

(0.3059113266001409, 0.3257581871738195, 0.13940529577028068)